In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import xarray as xr
import pandas as pd
import glob

In [2]:
### ================================================
### force time vector to be proper format and range
### ================================================
def make_dates(start=None, end=None):
    '''
    Creates a monthly DateTime64 vector centered on 15th each month
    
    Input
    =========
    start : str 'YYYY-MM'
    end :   str 'YYYY-MM'
            
    Output
    =========
    dates : dataArray 
    
    Example
    ==========
    dates = make_dates(start='1990-01', end='2017-12')
    '''
    dates = pd.date_range(start=f'{start}-01T00:00:00.000000000', 
                          end=f'{end}-01T00:00:00.000000000',
                          freq='MS')+ np.timedelta64(14, 'D')
    da = xr.DataArray(dates, dims='time')
    
    return da

In [3]:
def convert_to_datetime64(times, use_cftime):
    """
    Return times as np.datetime64[ns] or cftime.DatetimeProlepticGregorian
    depending on whether the dates fall within the inclusive bounds of 
    np.datetime64[ns]: [1678-01-01 AD, 2261-12-31 AD].

    Alternatively, always returns as cftime.DatetimeProlepticGregorian if
    `use_cf_time` is True.
    
    source : https://gitlab.com/deltares/imod/imod-python/commit/9b1ae0e9a51b0fefe6ef86c2a2f9ef988024b464
    """
    out_of_bounds = False
    if use_cftime:
        converted = [
            cftime.DatetimeProlepticGregorian(*time.timetuple()[:6]) for time in times
        ]
    else:
        for time in times:
            year = time.year
            if year < 1678 or year > 2261:
                out_of_bounds = True
                break

        if out_of_bounds:
            warnings.warn(
                "Dates are outside of np.datetime64[ns] timespan."
                "Converting to cftime.DatetimeProlepticGregorian."
            )
            converted = [
                cftime.DatetimeProlepticGregorian(*time.timetuple()[:6]) for time in times
            ]
        else:
            converted = [np.datetime64(time, "ns") for time in times]

    return converted

## Save pCO2

In [4]:
# load data
ds_mod = xr.open_dataset('/local/data/artemis/simulations/GCB/2020models/gcb_load_A_2020.nc', decode_times=False)

# only load these models
models = [ 'cesm_spco2_1x1_A', 
           'csiro_spco2_1x1_A',
           'fesom_spco2_1x1_A',
           'mpi_spco2_1x1_A', 
           'cnrm_spco2_1x1_A',
           'planktom_spco2_1x1_A',
           'noresm_spco2_1x1_A',
           'princeton_spco2_1x1_A',]

# only load variables need and change coord names
ds_mod = ds_mod[models]
ds_mod = ds_mod.rename({'T':'time','Y':'lat','X':'lon'})
ds_mod['time'] = make_dates(start='1958-01', end='2019-12')

# load ipsl model
ds_ipsl = xr.open_dataset('/local/data/artemis/workspace/gloege/data/GCB/IPSL_corrected.nc').rename({'months':'time'})
ds_ipsl['time'] = make_dates(start='1958-01', end='2019-12')
ds_ipsl['lon'] = list(map(lambda x: x+360 if x<0 else x, ds_ipsl['lon'].values))

# add corrected ipsl model
ds_mod['ipsl_spco2_1x1_A'] = ds_ipsl['IPSL_pco2_corrected'].transpose('time','lat','lon')

# select years
ds_mod = ds_mod.sel(time=slice('1982-01','2019-12'))

In [5]:
ds_mod.to_netcdf('/local/data/artemis/workspace/gloege/data/GCB/GCB2020-spco2_1x1_mon_198201-201912.nc')

In [9]:
ds_mod.lat

<xarray.DataArray 'lat' (lat: 180)>
array([-89.5, -88.5, -87.5, -86.5, -85.5, -84.5, -83.5, -82.5, -81.5, -80.5,
       -79.5, -78.5, -77.5, -76.5, -75.5, -74.5, -73.5, -72.5, -71.5, -70.5,
       -69.5, -68.5, -67.5, -66.5, -65.5, -64.5, -63.5, -62.5, -61.5, -60.5,
       -59.5, -58.5, -57.5, -56.5, -55.5, -54.5, -53.5, -52.5, -51.5, -50.5,
       -49.5, -48.5, -47.5, -46.5, -45.5, -44.5, -43.5, -42.5, -41.5, -40.5,
       -39.5, -38.5, -37.5, -36.5, -35.5, -34.5, -33.5, -32.5, -31.5, -30.5,
       -29.5, -28.5, -27.5, -26.5, -25.5, -24.5, -23.5, -22.5, -21.5, -20.5,
       -19.5, -18.5, -17.5, -16.5, -15.5, -14.5, -13.5, -12.5, -11.5, -10.5,
        -9.5,  -8.5,  -7.5,  -6.5,  -5.5,  -4.5,  -3.5,  -2.5,  -1.5,  -0.5,
         0.5,   1.5,   2.5,   3.5,   4.5,   5.5,   6.5,   7.5,   8.5,   9.5,
        10.5,  11.5,  12.5,  13.5,  14.5,  15.5,  16.5,  17.5,  18.5,  19.5,
        20.5,  21.5,  22.5,  23.5,  24.5,  25.5,  26.5,  27.5,  28.5,  29.5,
        30.5,  31.5,  32.5,  33.5,  34.5

## Save fgCO2

In [9]:
del ds_mod

In [10]:
# load data
ds_mod = xr.open_dataset('/local/data/artemis/simulations/GCB/2020models/gcb_load_A_2020.nc', decode_times=False)

# only load these models
models = [ 'cesm_fgco2_1x1_A', 
           'csiro_fgco2_1x1_A',
           'fesom_fgco2_1x1_A',
           'ipsl_fgco2_1x1_A',
           'mpi_fgco2_1x1_A', 
           'cnrm_fgco2_1x1_A',
           'planktom_fgco2_1x1_A',
           'noresm_fgco2_1x1_A',
           'princeton_fgco2_1x1_A',]

# only load variables need and change coord names
ds_mod = ds_mod[models]
ds_mod = ds_mod.rename({'T':'time','Y':'lat','X':'lon'})
ds_mod['time'] = make_dates(start='1958-01', end='2019-12')

# select years
ds_mod = ds_mod.sel(time=slice('1982-01','2019-12'))

In [11]:
ds_mod.to_netcdf('/local/data/artemis/workspace/gloege/data/GCB/GCB2020_fgco2_1x1_A.nc')